# A script that generate images in the output folder : non attacked, saliency non attacked, attacked, saliency attacked (with all combinaisons of : models, attacks, effect of attack)

In [13]:
#!pip install tf-keras-vis

In [14]:
print("Executing : Load libraries")

import os
# uncomment to force the non use of GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
from tf_keras_vis.utils import num_of_gpus

from keras.utils import load_img, img_to_array, array_to_img

import foolbox as fb
from foolbox.criteria import TargetedMisclassification
from foolbox.attacks import PGD

from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions

from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.saliency import Saliency

from PIL import Image, ImageChops, ImageEnhance

import numpy as np
import cv2
from scipy.stats import pearsonr
from skimage.metrics import structural_similarity as ssim

_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

path_project_root = os.path.dirname(os.path.abspath(''))

Executing : Load libraries
Tensorflow recognized 0 GPUs


# SALIENCY

In [15]:
def load_model(model_name="MobileNetV2"):
    """
        Return the selected model. Argument : name of the model.
    """
    print("Executing : Load Model")
    model=""
    if model_name=="VGG16":
        from keras.applications.vgg16 import VGG16 as Model
        model = Model(weights='imagenet', include_top=True)
    elif model_name=="MobileNetV2":
        from keras.applications.mobilenet_v2 import MobileNetV2 as Model
        model = Model(weights='imagenet', include_top=True)
    elif model_name=="ResNet152V2":
        from keras.applications.resnet_v2 import ResNet152V2 as Model
        model = Model(weights='imagenet', include_top=True)
    else:
        print("Error : model not available")
        raise NameError()
    return model

def get_image(path, size=(224,224), model_name="MobileNetV2"):
    print("Executing : Load And Preprocess Image")
    image= load_img(path)
    image = tf.cast(image, tf.float32)
    image = preprocess_image(image, model_name)
    return image

def preprocess_image(image, model_name):
    if model_name=="VGG16":
        image = tf.keras.applications.vgg16.preprocess_input(image)
    elif model_name=="MobileNetV2":
        image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    elif model_name=="ResNet152V2":
        image = tf.keras.applications.resnet_v2.preprocess_input(image)
    return image

def get_score_function(index_list):
    print("Executing : Create Score Function")
    return CategoricalScore(index_list)

def get_saliency_object(model):
    print("Executing : Create Saliency Object")
    replace2linear = ReplaceToLinear()
    saliency = Saliency(model, model_modifier=replace2linear, clone=True)
    return saliency

# ATTACK

In [16]:
def get_attacked_image_FGSM(image, model, image_index, debug = False, attack_rate = 15): #attack_rate entre 0 et 100

    pretrained_model = model
    pretrained_model.trainable = False

    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = image[None, ...]
  
    image_probs = pretrained_model.predict(image)
    label = tf.one_hot(image_index, image_probs.shape[-1])
    label = tf.reshape(label, (1, image_probs.shape[-1]))
    
    if debug:
      decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
      _, image_class, class_confidence = decode_predictions(image_probs, top=1)[0][0]
      plt.imshow(image[0] * 0.5 + 0.5)  # To change [-1, 1] to [0,1]
      plt.title('{} : {:.2f}% Confidence'.format(image_class, class_confidence*100))
      plt.show()
    

    loss_object = tf.keras.losses.CategoricalCrossentropy()

    with tf.GradientTape() as tape:
      tape.watch(image)
      prediction = pretrained_model(image)
      loss = loss_object(label, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    perturbations = signed_grad
    
    #for i, eps in enumerate(epsilons):
    adv_x = image + (attack_rate/100)*perturbations
    adv_x = tf.clip_by_value(adv_x, -1, 1)
    
    if debug :
      plt.imshow(perturbations[0] * 0.5 + 0.5);  # To change [-1, 1] to [0,1]
      plt.title("Noise")
      plt.show()
       #epsilons = [0, 0.01, 0.1, 0.15] #noter "taux d'attaque"
      descriptions = ['Epsilon = {:0.3f}'.format(attack_rate)]
      _, label, confidence = decode_predictions(pretrained_model.predict(adv_x), top=1)[0][0]
      plt.imshow(adv_x[0]*0.5+0.5)
      plt.title('{} \n {} : {:.2f}% Confidence'.format(descriptions[0], label, confidence*100))
      plt.show()

    attacked_image = adv_x[0]
    return attacked_image

def get_attacked_image_PGD(model, orig_input, debug = False, attack_rate = 10, class_target = 999) : #class_target c'est la classe que l'on cherche à appliquer
    orig_input = tf.expand_dims(orig_input, 0)
    
    fmodel = fb.TensorFlowModel(model, bounds=(-255, 255))
   
    #goldfish = 1; brown bear = 294; assault rifle = 413 #class de notre image
    # 2 = great white shark #classe que l'on cherche à obtenir
    adv_label = tf.convert_to_tensor([class_target])

    criterion = TargetedMisclassification(adv_label)

    attack = PGD()
    input_as_tensor = tf.convert_to_tensor(orig_input)
    adv_input = attack.run(fmodel, input_as_tensor, criterion, epsilon=attack_rate)
    adv_img = (adv_input.numpy() + 255) / 2
    adv_img = adv_img.reshape(224, 224, 3)
    adv_img = array_to_img(adv_img)
    b, g, r = adv_img.split()
    adv_img = Image.merge("RGB", (r, g, b))
    
    if debug :
      adv_input = img_to_array(adv_img)
      adv_input = adv_input.reshape((1, 224, 224, 3))
      adv_input = preprocess_input(adv_input)
      predictions = model.predict(adv_input)
      labels = decode_predictions(predictions)
      kind = labels[0][0][1].replace("_", " ").title()
      percent = round(labels[0][0][2] * 100, 2)
      print(f"This is a {kind}. I am {percent} % sure.")
      print()
      print("Other suggestions:")
      for i in range(4):
        kind = labels[0][i+1][1].replace("_", " ").title()
        percent = round(labels[0][i+1][2] * 100, 2)
        print(f"{kind}: {percent} %")
      difference = ImageChops.difference(adv_img, orig_input)
      plt.figure()
      plt.imshow(np.array(difference))
      plt.show()
      difference = ImageEnhance.Brightness(difference).enhance(10)
      plt.figure()
      plt.imshow(np.array(difference))
      plt.show()

    return adv_img

# MAIN PROGRAM

If 1 image is 100Ko => 40Mo generated for each input image

In [17]:
models_availables = ["ResNet152V2", "MobileNetV2", "VGG16"]
attacks_availables = ["FGSM", "PGD"]
input_directory  = os.path.join(path_project_root, 'final_dir\images\input')
output_directory = os.path.join(path_project_root, 'final_dir\images\output\\')
index_images_model = [404, 294, 500, 1, 576, 587, 606, 413]
epsilons = [0.0, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1.0]

score_list = list()
for value in index_images_model:
    score_list.append(get_score_function([value]))

Executing : Create Score Function
Executing : Create Score Function
Executing : Create Score Function
Executing : Create Score Function
Executing : Create Score Function
Executing : Create Score Function
Executing : Create Score Function
Executing : Create Score Function


In [18]:
for model_name in models_availables:
    model = load_model(model_name)
    saliency = get_saliency_object(model)
    for attack_name in attacks_availables:
        for attack_rate in epsilons:
            image_index = 0
            for filename in os.listdir(input_directory):
                filepath = os.path.join(input_directory, filename)
                if os.path.isfile(filepath):

                    original_image = get_image(filepath, model_name)
                    print("Executing : Saliency")
                    saliency_image = saliency(score_list[image_index], original_image)[0]


                    print("Executing : Attack " + attack_name)
                    attacked_image=""
                    if attack_name==attacks_availables[0]:
                        attacked_image = get_attacked_image_PGD(model, orig_input=original_image, attack_rate=attack_rate)
                    elif attack_name=="FGSM":
                        attacked_image = get_attacked_image_FGSM(image=original_image,image_index=index_images_model[image_index] , model=model, attack_rate=attack_rate)
                    
                    print("Executing : Saliency on Attacked image")
                    attacked_image_array_preprocessed = preprocess_image(np.array(attacked_image).astype(np.float32), model_name)
                    attacked_saliency_image = saliency(score_list[image_index], attacked_image_array_preprocessed)[0]

                    print("Executing : Prediction attacked image")
                    img_attacked_to_predict = np.resize(attacked_image_array_preprocessed, (1, 224, 224, 3))
                    predictions_attacked = model.predict(img_attacked_to_predict)
                    prediction_attacked = round(predictions_attacked[0][index_images_model[image_index]]*1000000)

                    original_image = cv2.imread(filepath)
                    img_original_to_predict = np.resize(original_image, (1, 224, 224, 3))
                    predictions_original = model.predict(img_original_to_predict)
                    prediction_original = round(predictions_original[0][index_images_model[image_index]]*1000000)

                    filename_output = filename[0:-4] + "_" + model_name + "_" + attack_name + "_" + str(attack_rate)


                    print("Executing : Save images")
                    save_original_image = np.array(original_image)
                    save_attacked_image = np.array(attacked_image)
                    save_saliency_image = 255*np.array(saliency_image)
                    save_attacked_saliency_image = 255*np.array(attacked_saliency_image)
                    cv2.imwrite(output_directory + filename_output + "_" + str(prediction_original) + "_nosaliency_noattacked" + ".png", save_original_image)
                    cv2.imwrite(output_directory + filename_output + "_" + str(prediction_attacked) + "_nosaliency_attacked"   + ".png", save_attacked_image)
                    cv2.imwrite(output_directory + filename_output + "_" + "-1"                     + "_saliency_noattacked"   + ".png", save_saliency_image)
                    cv2.imwrite(output_directory + filename_output + "_" + "-1"                     + "_saliency_attacked"     + ".png", save_attacked_saliency_image)
                    
                    image_index+=1
                    print("-----------------------------------------------------------------------------------------------------")

Executing : Load Model
Executing : Create Saliency Object
Executing : Load And Preprocess Image
Executing : Saliency
Executing : Attack FGSM
Executing : Saliency on Attacked image
Executing : Prediction attacked image
1/1 [==============================] - 2s 2s/step
Executing : Save images
-----------------------------------------------------------------------------------------------------
Executing : Load And Preprocess Image
Executing : Saliency
Executing : Attack FGSM
Executing : Saliency on Attacked image
Executing : Prediction attacked image
1/1 [==============================] - 0s 211ms/step
Executing : Save images
-----------------------------------------------------------------------------------------------------
Executing : Load And Preprocess Image
Executing : Saliency
Executing : Attack FGSM


KeyboardInterrupt: 